In [1]:
import numpy as np
import symmray as sr
import quimb.tensor as qtn

/home/sijingdu/anaconda3/envs/symmray_nqs/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
Lx = 4
Ly = 2
D = 4
seed = 42

In [3]:
edges = qtn.edges_2d_square(Lx, Ly)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)
site_info

{(0, 0): {'inds': ['b(0, 0)-(0, 1)', 'b(0, 0)-(1, 0)', 'k0,0'],
  'duals': [0, 0, 0],
  'shape': [4, 4, 2],
  'coordination': 2,
  'tags': ('I0,0',)},
 (0, 1): {'inds': ['b(0, 0)-(0, 1)', 'b(0, 1)-(1, 1)', 'k0,1'],
  'duals': [1, 0, 0],
  'shape': [4, 4, 2],
  'coordination': 2,
  'tags': ('I0,1',)},
 (1,
  0): {'inds': ['b(0, 0)-(1, 0)',
   'b(1, 0)-(1, 1)',
   'b(1, 0)-(2, 0)',
   'k1,0'], 'duals': [1, 0, 0, 0], 'shape': [4,
   4,
   4,
   2], 'coordination': 3, 'tags': ('I1,0',)},
 (1,
  1): {'inds': ['b(0, 1)-(1, 1)',
   'b(1, 0)-(1, 1)',
   'b(1, 1)-(2, 1)',
   'k1,1'], 'duals': [1, 1, 0, 0], 'shape': [4,
   4,
   4,
   2], 'coordination': 3, 'tags': ('I1,1',)},
 (2,
  0): {'inds': ['b(1, 0)-(2, 0)',
   'b(2, 0)-(2, 1)',
   'b(2, 0)-(3, 0)',
   'k2,0'], 'duals': [1, 0, 0, 0], 'shape': [4,
   4,
   4,
   2], 'coordination': 3, 'tags': ('I2,0',)},
 (2,
  1): {'inds': ['b(1, 1)-(2, 1)',
   'b(2, 0)-(2, 1)',
   'b(2, 1)-(3, 1)',
   'k2,1'], 'duals': [1, 1, 0, 0], 'shape': [4,
   4,
  

In [4]:
peps = qtn.TensorNetwork()
rng = np.random.default_rng(seed)
N_f = int(Lx * Ly / 2)
random_charge_string = np.zeros(Lx*Ly, dtype=int)
random_charge_string[:N_f] = 1
rng.shuffle(random_charge_string)

index = 0
for site, info in sorted(site_info.items()):

    # bond index charge distribution
    block_indices = [
        sr.BlockIndex({0: d // 2, 1: d // 2}, dual=dual)
        for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
    ]
    # physical index
    p = info['shape'][-1]
    block_indices.append(
        sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
    )

    # data = sr.Z2FermionicArray.random(
    #     block_indices,
    #     charge=0,
    #     seed=rng,
    # )
    data = sr.U1FermionicArray.random(
        block_indices,
        charge=random_charge_string[index],
        seed=rng,
        oddpos=index if random_charge_string[index] == 1 else None,
    )

    peps |= qtn.Tensor(
        data=data,
        inds=info["inds"],
        tags=info["tags"],
    )
    index += 1

# required to view general TN as an actual PEPS
for i, j in site_info:
    peps[f"I{i},{j}"].add_tag([f"X{i}", f"Y{j}"])

peps.view_as_(
    qtn.PEPS,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
    x_tag_id="X{}",
    y_tag_id="Y{}",
    Lx=Lx,
    Ly=Ly,
)

PEPS(tensors=8, indices=18, Lx=4, Ly=2, max_bond=4)

In [5]:
norm = peps.make_norm()
norm.contract()

633.5205347719796

In [6]:
norm.contract_boundary(
    max_bond=64,
    progbar=True,
    layer_tags=["BRA", "KET"],
)

contracted boundary, Lx=3, Ly=2: : 1it [00:00, 56.45it/s]


633.5205347719801

In [7]:
t = 1.0
V = 8.0
mu = 0.7

terms = {
    (sitea, siteb): sr.fermi_hubbard_spinless_local_array(
        t=t, V=V, mu=mu,
        symmetry="U1",
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in peps.gen_bond_coos()
}

In [8]:
ham = qtn.LocalHam2D(Lx, Ly, terms)

In [9]:
su = qtn.SimpleUpdateGen(peps, ham)

In [10]:
# cluster energies may not be accuracte yet
su.evolve(33, tau=0.3)
su.evolve(33, tau=0.1)
su.evolve(33, tau=0.03)
su.evolve(33, tau=0.01)

n=132, tau=0.0100, energy~-4.530259: 100%|##########| 33/33 [00:01<00:00, 25.04it/s]


In [11]:
gs = su.get_state()

In [12]:
from symmray.utils import set_debug
set_debug(False)
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=16,
)

(1,) (0,)
(5, 9) (10, 5)


ValueError: Error when contracting blocks for sector (2, -2): shape-mismatch for sum

In [14]:
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=32,
)

-1.6888319453881473

In [15]:
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=64,
)

-1.6888319453881473